In [1]:
import pandas as pd
import numpy as np

Загрузка данных

In [2]:
data = pd.read_csv('train.csv')

In [3]:
numeric_columns = np.array(data.columns[:190])
categorical_columns = np.array(data.columns[190:230])

Для построения модели выберем только те переменные, в которых есть хотя бы одно значение, не равное NaN.

In [4]:
not_null_numeric_columns = np.array(numeric_columns[np.array(pd.notnull(data[numeric_columns]).any())])

In [5]:
not_null_categorical_columns = np.array(categorical_columns[np.array(pd.notnull(data[categorical_columns]).any())])

Для упрощения модели выберем только те категориальные признаки, в которых не больше ста различных значений.

In [6]:
categorical_less_or_equal_100 = []
for var in not_null_categorical_columns:
    if len(data[var].unique()) <= 100:
        categorical_less_or_equal_100.append(var)
categorical_less_or_equal_100 = np.array(categorical_less_or_equal_100)
categorical_less_or_equal_100

array(['Var191', 'Var193', 'Var194', 'Var195', 'Var196', 'Var201',
       'Var203', 'Var205', 'Var206', 'Var207', 'Var208', 'Var210',
       'Var211', 'Var212', 'Var213', 'Var215', 'Var218', 'Var219',
       'Var221', 'Var223', 'Var224', 'Var225', 'Var226', 'Var227',
       'Var228', 'Var229'], 
      dtype='|S6')

In [7]:
data_numeric = data[not_null_numeric_columns]
data_categorical = data[categorical_less_or_equal_100]
y = data['label']

Заполним пропущенные значения.

In [8]:
data_numeric = data_numeric.fillna(0)
data_categorical = data_categorical.fillna('NA')

Применим к категориальным данным one-hot encoding с помощью DictVectorizer.

In [9]:
from sklearn.feature_extraction import DictVectorizer

In [10]:
dv = DictVectorizer(sparse = False)
data_categorical = dv.fit_transform(data_categorical.T.to_dict().values())

In [11]:
data_transformed = np.hstack((data_numeric, data_categorical))

In [12]:
from sklearn.cross_validation import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import precision_recall_curve, auc, roc_auc_score, precision_score, recall_score

Поделим выборку на обучающую и тестовую.

In [13]:
X_train, X_test, y_train, y_test = train_test_split(data_transformed, y, test_size = 0.3, random_state = 0)

Разобьём обучающую выборку на 3 фолда методом stratified k-fold.

In [15]:
skf = StratifiedKFold(y_train, 3, random_state = 0)

### Модель 1 : Линейная модель

In [14]:
from sklearn.linear_model import RidgeClassifier

Оценим качество модели на кросс-валидации.

In [16]:
ridge = RidgeClassifier()
scores = cross_val_score(ridge, X_train, y_train, cv = skf)
scores

array([ 0.92527119,  0.92540512,  0.92566301])

Видим, что метрика accuracy достаточно хорошая, однако классы несбалансированны, и она не очень показательна.

Обучим модель на всей обучающей выборке.

In [17]:
ridge.fit(X_train, y_train)
ridge_df = ridge.decision_function(X_test)
ridge_predictions = ridge.predict(X_test)

Оценим основную метрику качества - AUC-PRC.

In [19]:
precision, recall, thresholds = precision_recall_curve(y_test, ridge_df)
auc(recall, precision)

0.073479126464954858

Она совсем небольшая - модель плохая.

Оценим модель с помощью дополнительных метрик качества.

AUC-ROC

In [20]:
roc_auc_score(y_test, ridge_df)

0.50200014580000085

Precision

In [21]:
precision_score(y_test, ridge_predictions)

0.0

Recall

In [22]:
recall_score(y_test, ridge_predictions)

0.0

Они также говорят о том, что модель плохая.

### Модель 2 : Случайный лес

In [23]:
from sklearn.ensemble import RandomForestClassifier

Оценим качество модели на кросс-валидации.

In [24]:
rf = RandomForestClassifier()
scores = cross_val_score(rf, X_train, y_train, cv = skf)
scores

array([ 0.92567296,  0.92567296,  0.92606483])

Видим, что метрика accuracy достаточно хорошая, однако классы несбалансированны, и она не очень показательна.

Обучим модель на всей обучающей выборке.

In [25]:
rf.fit(X_train, y_train)
rf_probabilities = np.array([x[1] for x in rf.predict_proba(X_test)])
rf_predictions = rf.predict(X_test)

Оценим основную метрику качества - AUC-PRC.

In [26]:
precision, recall, thresholds = precision_recall_curve(y_test, rf_probabilities)
auc(recall, precision)

0.074041828845549967

Она совсем небольшая - модель плохая.

Оценим модель с помощью дополнительных метрик качества.

AUC-ROC

In [27]:
roc_auc_score(y_test, rf_probabilities)

0.49059712773998487

Precision

In [28]:
precision_score(y_test, rf_predictions)

0.0

Recall

In [29]:
recall_score(y_test, rf_predictions)

0.0

Они также говорят о том, что модель плохая.

### Модель 3 : Градиентный бустинг

In [36]:
from sklearn.ensemble import GradientBoostingClassifier

Оценим качество модели на кросс-валидации.

In [30]:
gb = GradientBoostingClassifier()
scores = cross_val_score(gb, X_train, y_train, cv = skf)
scores

array([ 0.92419981,  0.92500335,  0.92472542])

Видим, что метрика accuracy достаточно хорошая, однако классы несбалансированны, и она не очень показательна.

Обучим модель на всей обучающей выборке.

In [31]:
gb.fit(X_train, y_train)
gb_probabilities = np.array([x[1] for x in gb.predict_proba(X_test)])
gb_predictions = gb.predict(X_test)

Оценим основную метрику качества - AUC-PRC.

In [32]:
precision, recall, thresholds = precision_recall_curve(y_test, gb_probabilities)
auc(recall, precision)

0.080087771623378129

Она незначительно улучшилась по сравнению с предыдущими моделями, но всё равно совсем маленькая.

Оценим модель с помощью дополнительных метрик качества.

AUC-ROC

In [33]:
roc_auc_score(y_test, gb_probabilities)

0.50836116962295352

Precision

In [34]:
precision_score(y_test, gb_predictions)

0.14285714285714285

Recall

In [35]:
recall_score(y_test, gb_predictions)

0.0013605442176870747

Precision и recall стали чуть лучше, но над моделью всё равно ещё нужно работать.